In [1]:
import os, time
os.environ["MUJOCO_GL"] = "egl"
os.environ["EGL_PLATFORM"] = "surfaceless"
os.environ["LD_LIBRARY_PATH"] = "/usr/lib/nvidia:" + os.environ.get("LD_LIBRARY_PATH", "")

In [5]:
import gymnasium as gym
env = gym.make("FrankaKitchen-v1")
print("env obs space:", env.observation_space)
print("env act space:", env.action_space)


env obs space: Dict('achieved_goal': Dict('bottom burner': Box(-inf, inf, (2,), float64), 'hinge cabinet': Box(-inf, inf, (2,), float64), 'kettle': Box(-inf, inf, (7,), float64), 'light switch': Box(-inf, inf, (2,), float64), 'microwave': Box(-inf, inf, (1,), float64), 'slide cabinet': Box(-inf, inf, (1,), float64), 'top burner': Box(-inf, inf, (2,), float64)), 'desired_goal': Dict('bottom burner': Box(-inf, inf, (2,), float64), 'hinge cabinet': Box(-inf, inf, (2,), float64), 'kettle': Box(-inf, inf, (7,), float64), 'light switch': Box(-inf, inf, (2,), float64), 'microwave': Box(-inf, inf, (1,), float64), 'slide cabinet': Box(-inf, inf, (1,), float64), 'top burner': Box(-inf, inf, (2,), float64)), 'observation': Box(-inf, inf, (59,), float64))
env act space: Box(-1.0, 1.0, (9,), float64)


In [4]:
import gymnasium as gym
import gymnasium_robotics
from gymnasium.wrappers import RecordVideo
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output

gym.register_envs(gymnasium_robotics)

save_dir = f"/home/xli990/paichichi/videos/franka/reach_2views/{int(time.time())}"

env = gym.make(
    "FrankaKitchen-v1",
    tasks_to_complete=["microwave", "kettle"],
    render_mode="rgb_array",
)

# 录制：每个 episode 都录（你也可以改成只录第一个）
env = RecordVideo(
    env,
    video_folder=save_dir,
    episode_trigger=lambda episode_id: True,
    name_prefix="franka_kitchen",
)

obs, info = env.reset(seed=42)
env.action_space.seed(42)

try:
    for t in range(50):
        a = env.action_space.sample()
        obs, r, term, trunc, info = env.step(a)

        d_micro = np.linalg.norm(obs["achieved_goal"]["microwave"] - obs["desired_goal"]["microwave"])
        d_kettle = np.linalg.norm(obs["achieved_goal"]["kettle"] - obs["desired_goal"]["kettle"])

        print(t, "r=", r, "d_micro=", float(d_micro), "d_kettle=", float(d_kettle),
              "done=", info["episode_task_completions"])
        if term or trunc:
            break
finally:
    env.close()

print("Videos saved to:", save_dir)

Videos saved to: /home/xli990/paichichi/videos/franka/reach_2views/1770039724
